In [ ]:
# Uncomment this cell to run prompt optimization on a study. Most recent runs require running on a server.
# !uv run python -m syftr.prompt_optimization --study-config studies/rank1--rag-and-agents--drdocs_hf.yaml

In [ ]:
# This cell takes two studies (witout PO and with it and compares the result). Please, note commented code:
# PO on studies/rank1--rag-and-agents--drdocs_hf.yaml weren't run on pareto, so we need all trials to perform a join, 
# not only Pareto trials.

import pandas as pd
import optuna
from syftr.optuna_helper import get_pareto_mask
from syftr.configuration import cfg

RAW_STUDY = "rank1--rag-and-agents--drdocs_hf"
PO_STUDY = "rank1--rag-and-agents--drdocs_hf_prompt_optimization"

df = optuna.load_study(
    study_name=RAW_STUDY, storage=cfg.database.get_optuna_storage()
).trials_dataframe()
# df = df[df["values_0"] > 0.5]
# pareto_mask = get_pareto_mask(df)
# df = df[get_pareto_mask]
df["study_name"] = RAW_STUDY

optimized_df = optuna.load_study(
    study_name=PO_STUDY, storage=cfg.database.get_optuna_storage()
).trials_dataframe()
optimized_df = optimized_df.drop_duplicates(
    subset=["user_attrs_parent_number"], keep="last"
)
optimized_df["study_name"] = PO_STUDY

results = pd.merge(
    df,
    optimized_df,
    left_on="number",
    right_on="user_attrs_parent_number",
)
results["Cost mult, x"] = results["values_1_y"] / results["values_1_x"]
results["Acc. improvement"] = (results["values_0_y"] - results["values_0_x"]) * 100
results = results.rename(
    columns={
        "values_0_x": "Accuracy pre-opt",
        "values_1_x": "Cost pre-opt",
        "values_0_y": "Accuracy post-opt",
        "values_1_y": "Cost post-opt",
    }
)
results["Accuracy pre-opt"] *= 100
results["Accuracy post-opt"] *= 100
results = results[
    [
        "Accuracy pre-opt",
        "Cost pre-opt",
        "Accuracy post-opt",
        "Cost post-opt",
        "Acc. improvement",
        "Cost mult, x",
        "user_attrs_flow_name",
        "user_attrs_parent_number",
    ]
]
results

In [ ]:
optimized_df["user_attrs_optimized_dataset_description"].tolist()